In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

- Colonne status merge avec colonne criticality 

==> 4 criticitées :     
- IDLE
- INFO
- WARNING
- ERROR

## <center>**Import du Dataset originel**<center>

In [8]:
df = pd.read_csv("../data/metrics/Arthur_dataset_V2.csv", index_col=0)
df.head(2)

,source_events,timestamp,criticality_events,identification_events,type_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil,Total Pages Counter,22881,Viktor,Operator,36192,100000
1,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil,Foiled Pages Counter,31092,Viktor,Operator,36192,100000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510431 entries, 0 to 3510430
Data columns (total 11 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   source_events              object
 1   timestamp                  object
 2   criticality_events         object
 3   identification_events      int64 
 4   type_modules               object
 5   name_counters_modules      object
 6   value_counters_modules     int64 
 7   name_connected_operators   object
 8   level_connected_operators  object
 9   varnishLevelsTargetvolume  int64 
 10  varnishLevelsTotalvolume   int64 
dtypes: int64(4), object(7)
memory usage: 321.4+ MB


## <center>**Encodage du Dataset**<center>

In [10]:
df.criticality_events.unique()

array(['INFO', 'WARNING', 'ERROR', 'IDLE'], dtype=object)

In [11]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
criticality = {'IDLE': 0, 'INFO': 1, 'WARNING': 2, 'ERROR':3}
df.criticality_events.replace(criticality, inplace=True)
df.criticality_events = pd.to_numeric(df.criticality_events).astype('int64')

In [12]:
object_columns = ["source_events","type_modules","name_counters_modules","name_connected_operators","level_connected_operators"]

num_columns = ["identification_events", "value_counters_modules", "varnishLevelsTargetvolume", "varnishLevelsTotalvolume"]

In [13]:
df = df.drop("timestamp", axis= 1)

In [14]:
df_encoded = pd.get_dummies(df, columns= object_columns, drop_first=True)

In [15]:
df_encoded.info()
df_encoded.head(2)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510431 entries, 0 to 3510430
Data columns (total 25 columns):
 #   Column                                      Dtype
---  ------                                      -----
 0   criticality_events                          int64
 1   identification_events                       int64
 2   value_counters_modules                      int64
 3   varnishLevelsTargetvolume                   int64
 4   varnishLevelsTotalvolume                    int64
 5   source_events_ICB n°2                       uint8
 6   source_events_ICB n°4                       uint8
 7   source_events_ICB n°5                       uint8
 8   source_events_ICB n°6                       uint8
 9   source_events_ICB n°7                       uint8
 10  source_events_ICB n°8                       uint8
 11  source_events_Kernel                        uint8
 12  source_events_PLC                           uint8
 13  source_events_Pilot                         uint8
 14  so

,criticality_events,identification_events,value_counters_modules,varnishLevelsTargetvolume,varnishLevelsTotalvolume,source_events_ICB n°2,source_events_ICB n°4,source_events_ICB n°5,source_events_ICB n°6,source_events_ICB n°7,...,source_events_RCB n°2,source_events_RCB n°3,source_events_iFoil,type_modules_iFoil,name_counters_modules_Foiled Pages Counter,name_counters_modules_Total Pages Counter,name_connected_operators_JAN,name_connected_operators_Micka,name_connected_operators_Viktor,level_connected_operators_Operator
0,1,391,22881,36192,100000,0,0,0,0,0,...,0,0,1,1,0,1,0,0,1,1
1,1,391,31092,36192,100000,0,0,0,0,0,...,0,0,1,1,1,0,0,0,1,1


In [16]:
X = df_encoded.drop("criticality_events", axis= 1)
y = df_encoded.criticality_events

In [17]:
X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.25, random_state=40000)

In [18]:
Norm_Scaler = MinMaxScaler()

X_train[num_columns]= Norm_Scaler.fit_transform(X_train[num_columns])
X_test[num_columns]= Norm_Scaler.transform(X_test[num_columns])

In [19]:
rf_model = RandomForestClassifier(n_estimators=1000, random_state=40000)

rf_model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000, random_state=40000)

In [20]:
y_pred = rf_model.predict(X_test)

accu = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [21]:
print(accu)


0.9067237308684515


In [22]:
print(conf_matrix)

[[532779      0  22573   9885]
 [     1  16053    309    746]
 [ 21780    417 157944   4878]
 [ 12779    888   7604  88972]]


In [23]:
print(class_report)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94    565237
           1       0.92      0.94      0.93     17109
           2       0.84      0.85      0.85    185019
           3       0.85      0.81      0.83    110243

    accuracy                           0.91    877608
   macro avg       0.89      0.89      0.89    877608
weighted avg       0.91      0.91      0.91    877608

